In [61]:
import pandas as pd
import numpy as np

In [62]:
education = pd.read_csv('../data/raw/Education.csv')
education.head()

,Табельный номер руководителя,id,Вид образования,Специальность
0,NaN,ОРГ1-00131,Высшее образование - бакалавриат,Юриспруденция
1,NaN,ОРГ1-03220,Среднее профессиональное образование,Финансы
2,NaN,ОРГ1-03008,Высшее образование,Юриспруденция
3,NaN,ОРГ1-00922,Высшее образование - бакалавриат,Бакалавр технических наук
4,NaN,ОРГ1-00922,Высшее образование,Машины и аппараты химических производств и пре...


In [63]:
# map for level of education:
education_level_map = {
    'Начальное общее образование': 1,
    'Среднее общее образование':2,
    'Среднее (полное) общее образование':3,
    'Основное общее образование':3,
    'Начальное профессиональное образование':4,
    'Среднее профессиональное образование':5,
    'Профессиональное обучение':6,
    'Неполное высшее образование':7,
    'Высшее образование':8,
    'Высшее образование - бакалавриат':8,
    'Высшее образование - специалитет, магистратура':9,
    'Повышение квалификации':10,
    'Послевузовское образование':10,
    'Дополнительное профессиональное образование':11,
    'Переподготовка':12,
    'Аспирантура':13
}

len(education_level_map)

16

In [64]:
education['edu_level'] = education['Вид образования'].replace(education_level_map)

In [65]:
appearences_in_education = education.id.value_counts().reset_index()
appearences_in_education.columns = ['id', 'count_appearence']
education = pd.merge(education, appearences_in_education, how='left', on='id')

In [66]:
education = education.sort_values(by=['id','edu_level']).dropna(subset='edu_level')
education = education.drop_duplicates(subset='id', keep='last')

In [67]:
education_chief = education[['Табельный номер руководителя', 'id']].copy()
education_chief.drop_duplicates(inplace=True)
education_chief = education_chief['Табельный номер руководителя'].value_counts().reset_index()
education_chief.columns = ['id', 'count_chiefed']
education_chief['count_chiefed'].fillna(0, inplace=True)
education_chief

,id,count_chiefed
0,ОРГ2-09100,1786
1,ОРГ2-02987,72
2,ОРГ1-00457,34
3,ОРГ1-00483,29
4,ОРГ2-01170,29
...,...,...
147,ОРГ1-02592,1
148,ОРГ1-02703,1
149,ОРГ1-03049,1
150,ОРГ1-00657,1


In [68]:
education = pd.merge(education, education_chief, on='id', how='left')

In [69]:
education['is_chief'] = np.where(education['count_chiefed']>0, 1, 0)

In [70]:
education.drop(['Табельный номер руководителя'], axis=1, inplace=True)

In [74]:
education['count_chiefed'].fillna(0, inplace=True)

In [75]:
education

,id,Вид образования,Специальность,edu_level,count_appearence,count_chiefed,is_chief
0,ОРГ1-00001,Повышение квалификации,Организация подготовки проектной документации,10.0,3,0.0,0
1,ОРГ1-00004,Повышение квалификации,NaN,10.0,6,2.0,1
2,ОРГ1-00005,Повышение квалификации,Безопасность строительства и осуществление стр...,10.0,6,4.0,1
3,ОРГ1-00010,Высшее образование,Экономика и организация машиностроительной про...,8.0,1,0.0,0
4,ОРГ1-00013,Высшее образование,Финансы и кредит,8.0,3,0.0,0
...,...,...,...,...,...,...,...
4754,ОРГ2-09282,"Высшее образование - специалитет, магистратура",Автомобили и автомобильное хозяйство,9.0,1,0.0,0
4755,ОРГ2-09283,Высшее образование,Бухгалтерский учет и аудит,8.0,1,0.0,0
4756,ОРГ2-09284,Начальное профессиональное образование,"Слесарь по ремонту автомобилей, электромонтер ...",4.0,1,0.0,0
4757,ОРГ2-09285,Высшее образование,Сооружение и эксплуатация газонефтепроводов и ...,8.0,1,0.0,0


In [76]:
education['exists_in_education'] = 1

In [77]:
education = education.drop_duplicates(subset='id')

In [79]:
education.to_csv('../data/prepared/education.csv', index=False)